In [ ]:
%load_ext autoreload
%autoreload 2
from arpes.all import *
import os.path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
%matplotlib inline

In [ ]:
# Define directory
path = os.path.abspath(r'C:\Users\hellbrue\Documents\Bi2212')

# Different parameters used to set up scans at SLS
scan_type = '*'
direction = 'X'
phi = '*'
energy = 'eV'
polarization = '*'
temp = '*'
e_pass = '*'+'eVpass'
comment = '*'
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

au_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
au_files = glob.glob(au_full_file)
print(len(au_files))
au_files

In [324]:
au_raw_data_list = {}
for file in au_files:
    #data_list.append(load_data(file, location="ULTRA"))
    data = load_data(file, location="ULTRA")
    # k_data = convert_to_kspace(data.spectrum)
    file_name = file.replace(path, '')[1:-3]
    au_raw_data_list.update({file_name : data})

poly Au used to determine Fermi level. Determine Fermiedge and apply to data below

In [ ]:
au_lt = au_raw_data_list['EDC_X_M0_eV_CIRCP_LT_25eVpass_GoldRef 3_0000']
au_lt.S.plot()

In [ ]:
from arpes.analysis.general import rebin

binned = rebin(au_lt, phi=10).spectrum
binned.plot()

In [ ]:
from arpes.all import AffineBroadenedFD, broadcast_model
fermi_level = binned.sel(phi=slice(-0.2,0.2), eV=slice(-0.2,0.1))
results = broadcast_model(AffineBroadenedFD, fermi_level, "phi")

fig, ax = plt.subplots()
fermi_level.plot(ax=ax)
ax.scatter(*results.F.p("fd_center").G.to_arrays(), color="red")
ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
from arpes.fits.fit_models import LinearModel
lin_mod = LinearModel().guess_fit(results.F.p("fd_center"))
lin_mod.plot()
plt.show()

In [ ]:
edc = au_lt.spectrum
edge = lin_mod.eval(x=edc.phi)

corrected_edc = edc.G.shift_by(edge, "eV")

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
corrected_edc.plot(ax=axes[0], vmax=100)
edc.plot(ax=axes[1], vmax=200)
# plot vertical lines for y = 0
for ax in axes:
    ax.set_title("")
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
# Define directory
path = os.path.abspath(r'C:\Users\hellbrue\Documents\Bi2212')

# Different parameters used to set up scans at SLS
scan_type = 'FS'
direction = 'AN'
phi = 'P16p5'
energy = '*'
polarization = '*'
temp = '*'
e_pass = '*'+'eVpass'
comment = ''
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

bi2223_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
bi2223_files = glob.glob(bi2223_full_file)
print(len(bi2223_files))
bi2223_files

In [332]:
# values for offset beta = 0.03
kp8 = np.array([-0.410, -0.410, -0.410, -0.410, -0.410])
kp9 = np.array([-0.452, -0.452, -0.452, -0.452, -0.452])
kp10 = np.array([-0.494, -0.494, -0.494, -0.494, -0.494])
kp11 = np.array([-0.535, -0.535, -0.535, -0.535, -0.535])
kp_list = np.array([kp8, kp9, kp10, kp11])
kps = abs(kp_list.T[0])

kx8_fit = np.array([0.439, 0.438, 0.437, 0.436, 0.435])
kx9_fit = np.array([0.396, 0.393, 0.39 , 0.387, 0.384])
kx10_fit = np.array([0.371, 0.365, 0.359, 0.353, 0.347])
kx11_fit = np.array([0.347, 0.331, 0.315, 0.299, 0.283])
kx_fit_list = np.array([kx8_fit, kx9_fit, kx10_fit, kx11_fit])
kx_fit_fl = kx_fit_list.T

In [591]:
bi2223_data_list = {}
for file in bi2223_files:
    #data_list.append(load_data(file, location="ULTRA"))
    raw_data = load_data(file, location="ULTRA")
    edge = lin_mod.eval(x=raw_data.spectrum.phi)
    corrected_data = raw_data.spectrum.G.shift_by(edge, "eV")
    corrected_data.S.apply_offsets({"beta": -0.003, "chi": -0.79, "phi": -0.007})
    binned = rebin(corrected_data, phi=2, beta=2, eV=2)
    k_data = convert_to_kspace(binned)
    file_name = file.replace(path, '')[1:-3]
    bi2223_data_list.update({file_name : k_data})

Initalize datasets

In [592]:
lt_an_p20 = bi2223_data_list['FS_AN_P16p5_25eV_CIRCP_LT_20eVpass_0000']


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_an_p20.sel(eV=slice(-0.1,0.1), kx=slice(0,0.6)).sum("eV").plot(ax=ax, cmap='viridis')

plt.title('')
plt.tick_params(axis='both', which='major', labelsize=16)

for kx, ky in zip(kx_fit_fl[0], kp_list.T[0]):
    plt.scatter(kx/2, ky, color='red', s=50, marker='x')
    plt.scatter(kx/2, -ky, color='red', s=50, marker='x')
    plt.scatter(-kx/2, ky, color='red', s=50, marker='x')
    plt.scatter(-kx/2, -ky, color='red', s=50, marker='x')

ax.axhline(y=-0.2, color='k', linestyle='--', linewidth=1)
ax.axhline(y=0.2, color='k', linestyle='--', linewidth=1)
ax.axvline(x=-0.2, color='k', linestyle='--', linewidth=1)
ax.axvline(x=0.2, color='k', linestyle='--', linewidth=1)

ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
# set x and y axis to be equal
# plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(-0.58, 0.58)
# plt.ylim(0, 0.54)
plt.show()

In [594]:
import copy

symm_set = copy.deepcopy(lt_an_p20.sel(ky=slice(-0.55,0.55), kx=slice(0,0.6)))

symm_set_adj = symm_set.assign_coords(ky=np.round(symm_set.coords["ky"], 2))

# Assign negative kx and ky values for symmetry
data_inverted_x = symm_set_adj.assign_coords(kx=-symm_set_adj.coords["kx"])
data_inverted_xy = symm_set_adj.assign_coords(ky=-symm_set_adj.coords["ky"], kx=-symm_set_adj.coords["kx"])


sel_neg = data_inverted_xy.sel(kx=slice(-0.6,-0.0,-1), ky=slice(-0.55,0.55,-1))
sel_pos = symm_set_adj.sel(kx=slice(0.0,0.6), ky=slice(-0.55,0.55))

boundary_region = 5  # Number of points to average around the boundary

# Take the last few points of data_part1 and first few points of data_part2
data_part1_end = sel_neg.isel(kx=slice(-boundary_region, None))
data_part2_start = sel_pos.isel(kx=slice(0, boundary_region))

# Compute the average
transition_data = 0.5 * (data_part1_end + data_part2_start)

# stitched_data = xr.concat([sel_neg, transition_data, sel_pos], dim="kx")
stitched_data = xr.concat([sel_neg.isel(kx=slice(None, -boundary_region)), 
                           transition_data, 
                           sel_pos.isel(kx=slice(boundary_region, None))], dim='kx')

common_kx = np.linspace(stitched_data['kx'].min(), stitched_data['kx'].max(), num=300)
common_ky = np.linspace(stitched_data['ky'].min(), stitched_data['ky'].max(), num=300)

# Interpolate both datasets onto the common grid
stitched_data_interp = stitched_data.interp(kx=common_kx, ky=common_ky)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
stitched_data.sel(eV=slice(-0.05,0.05)).sum("eV").plot(ax=ax, cmap='BuPu_r', vmin=200, vmax=2500, add_colorbar=False)

ax.set_title('')
ax.set_xlabel(r'k$_x$ ($\AA^{-1}$)', fontsize=16)
ax.set_ylabel(r'k$_y$ ($\AA^{-1}$)', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
plt.gca().set_aspect('equal', adjustable='box')
plt.ylim(-0.6, 0.6)
plt.xlim(-0.6, 0.6)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
stitched_data.sel(eV=slice(-0.05,0.05), ky=slice(-0.54,0.54)).sum("eV").plot(ax=ax, cmap='BuPu_r', vmin=200, vmax=2500, add_colorbar=False)

for kx, ky in zip(kx_fit_fl[0], kp_list.T[0]):
    ax.scatter(kx/2, ky, color='k', s=50, marker='x')
    ax.scatter(kx/2, -ky, color='k', s=50, marker='x')
    ax.scatter(-kx/2, ky, color='k', s=50, marker='x')
    ax.scatter(-kx/2, -ky, color='k', s=50, marker='x')
for ky, kx in zip(kx_fit_fl[0], kp_list.T[0]):
    ax.scatter(kx, ky/2, color='k', s=50, marker='x')
    ax.scatter(kx, -ky/2, color='k', s=50, marker='x')
    ax.scatter(-kx, ky/2, color='k', s=50, marker='x')
    ax.scatter(-kx, -ky/2, color='k', s=50, marker='x')

ax.set_title('')
ax.set_xlabel(r'k$_x$ ($\AA^{-1}$)', fontsize=16)
ax.set_ylabel(r'k$_y$ ($\AA^{-1}$)', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
plt.gca().set_aspect('equal', adjustable='box')

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
stitched_data.sel(eV=slice(-0.05,0.05), ky=slice(-0.54,0.54)).sum("eV").plot(ax=ax, cmap='BuPu_r', vmin=200, vmax=2500, add_colorbar=False)

plt.title('')
plt.tick_params(axis='both', which='major', labelsize=16)
cmap = plt.get_cmap('RdBu_r')
colors = [cmap(i) for i in np.linspace(0, 1, 10)]
del colors[4:7]
del colors[0]
del colors[-1]

i=0
for kx, ky in zip(kx_fit_fl, kp_list.T):
    plt.scatter(kx/2, ky, color=colors[i], s=50, marker='x')
    plt.scatter(kx/2, -ky, color=colors[i], s=50, marker='x')
    plt.scatter(-kx/2, ky, color=colors[i], s=50, marker='x')
    plt.scatter(-kx/2, -ky, color=colors[i], s=50, marker='x')
    i+=1
i=0
for ky, kx in zip(kx_fit_fl, kp_list.T):
    plt.scatter(kx, ky/2, color=colors[i], s=50, marker='x')
    plt.scatter(kx, -ky/2, color=colors[i], s=50, marker='x')
    plt.scatter(-kx, ky/2, color=colors[i], s=50, marker='x')
    plt.scatter(-kx, -ky/2, color=colors[i], s=50, marker='x')
    i+=1

cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
# set x and y axis to be equal
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(-0.54, 0.54)
plt.ylim(-0.58, 0.58)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_an_p20.sel(eV=slice(-0.05,0.05)).sum("eV").plot(ax=ax, cmap='BuPu_r', vmin=200, vmax=2500, add_colorbar=False)

plt.title('')
plt.tick_params(axis='both', which='major', labelsize=16)
cmap = plt.get_cmap('RdBu_r')
colors = [cmap(i) for i in np.linspace(0, 1, 10)]
del colors[4:7]
del colors[0]
del colors[-1]

i=0
for kx, ky in zip(kx_fit_fl, kp_list.T):
    plt.scatter(-kx/2, -ky, color=colors[i], s=50, marker='x')
    plt.scatter(ky, kx/2, color=colors[i], s=50, marker='x')
    i+=1

cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
# set x and y axis to be equal
# plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(-0.54, 0)
plt.ylim(0, 0.57)
ax.set_title('')
ax.set_xlabel(r'k$_x$ ($\AA^{-1}$)', fontsize=16)
ax.set_ylabel(r'k$_y$ ($\AA^{-1}$)', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.show()

Overview of the LT P5 Dataset

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_an_p20.sel(eV=slice(-0.02,0.02)).sum("eV").plot(ax=ax, cmap='viridis')

# plt.ylim(-0.56, 0)
# plt.xlim(-0, 0.56)
plt.title('')
# plt.ylabel('$k_y$ ($\AA^{-1}$)', fontsize=18)
# plt.xlabel('$k_x$ ($\AA^{-1}$)', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)
# plt.yticks(np.arange(-0.4, 0.5, 0.2))
# plt.xticks(np.arange(-0.4, 0.5, 0.2))

kys = [0.55, 0.5, 0.45, 0.4, 0.35, 0.30, 0.25, 0.22, 0.2]
kxs = [0.2, 0.21, 0.23, 0.26, 0.3, 0.35, 0.4, 0.45, 0.51]
for kx, ky in zip(kxs, kys):
    plt.scatter(ky, kx, color='red', s=50, marker='x')

ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.set_ticks([])
# set x and y axis to be equal
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [15]:
def normalize(data):
    return (data - data.sel(eV=slice(0.05,0.07)).mean("eV")) / max(data.values)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,5))

cmap = plt.get_cmap('copper')
colors = [cmap(i) for i in np.linspace(0, 1, 5)]

edc_an = lt_an_p20.sel(eV=slice(-0.74,0.1), kx=slice(0.51,0.59), ky=slice(0.16,0.23)).sum("kx").sum("ky")
edc_n = lt_an_p20.sel(eV=slice(-0.74,0.1), ky=slice(0.27,0.33), kx=slice(0.31,0.39)).sum("kx").sum("ky")
au_ref =binned.sel(eV=slice(-0.74,0.1), phi=slice(-0.05,0.05)).sum("phi")

normalize(au_ref).plot(ax=ax, label='Gold Ref', marker='o', markeredgewidth=1, markerfacecolor='none', markeredgecolor=colors[1], color=colors[1])
normalize(edc_n).plot(ax=ax, label='Node', marker='o', markeredgewidth=1, markerfacecolor='none', markeredgecolor=colors[2], color=colors[2])
normalize(edc_an).plot(ax=ax, label='Antinode', marker='o', markeredgewidth=1, markerfacecolor='none', markeredgecolor=colors[3], color=colors[3])

ax.set_xlim(-0.24,0.1)
ax.set_ylim(-0.05,1.1)
ax.set_title('')
ax.set_ylabel('Intensity (arb. units)', fontsize=18)
ax.set_xlabel('Binding Energy E$_B$ (eV)', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=16)

ax.axvline(x=-0.03, color='k', linestyle='--', linewidth=1)
plt.legend(fontsize=12)
plt.savefig('Bi2212_EDC.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2.5, 5))

cmap = plt.get_cmap('copper')
colors = [cmap(i) for i in np.linspace(0, 1, len(kxs)+3)]

i=0
for kx, ky in zip(kxs, kys):
    edc = lt_an_p20.sel(eV=slice(-0.1,0.1), kx=slice(kx-0.01,kx+0.01), ky=slice(ky-0.01,ky+0.01)).sum("kx").sum("ky")
    (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).plot(ax=ax, label=f'({kx}, {ky})', color=colors[i+1])
    ax.plot(-1*edc.sel(eV=slice(-0.08,0)).eV.values, (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).values, color=colors[i+1])
    i+=1

ax.set_title('')
ax.set_xlim(-0.08, 0.08)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_ylabel('Intensity', fontsize=18)
ax.set_xlabel('E$_B$ (eV)', fontsize=18)
e=0.025
ax.axvline(x=e, color='k', linestyle='--', linewidth=1)
ax.axvline(x=-e, color='k', linestyle='--', linewidth=1)

plt.savefig('Bi2212_gap.pdf', bbox_inches='tight')